<a href="https://colab.research.google.com/github/edith254-alt/Musila/blob/main/Copy_of_Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# === Base paths ===
sample_folder = r"C:\Users\DELL\Desktop\2 NIR IMAGES\2 uninfected sample  images\sample 2 images\sample images"
ref_folder    = r"C:\Users\DELL\Desktop\2 NIR IMAGES\2 uninfected sample  images\sample 2 images\reference images"

# === Output path for the absorbance map plots (without colorbar) ===
output_plot_folder = r"C:\Users\DELL\Desktop\2 NIR IMAGES\2 uninfected sample  images\sample 2 images\absorbance maps" # Renamed folder for clarity
os.makedirs(output_plot_folder, exist_ok=True) # Create the output folder if it doesn't exist

# === Wavelengths to process ===
wavelengths = "405nm, 430nm, 645nm, 660nm, 670nm, 680nm, 800nm, 850nm"
wavelength_list = [w.strip() for w in wavelengths.split(",")]

# === Loop through each wavelength ===
for wavelength in wavelength_list:
    print(f"\n🔍 Processing: {wavelength}")

    # Construct correct paths
    sample_path = os.path.join(sample_folder, wavelength, "image_1.png")
    ref_path    = os.path.join(ref_folder, wavelength, "image_1.png")

    # Load images
    sample_color = cv2.imread(sample_path)
    ref_color    = cv2.imread(ref_path)

    if sample_color is None or ref_color is None:
        print(f"❌ Could not load images for {wavelength}")
        print(f"    Sample path: {sample_path}")
        print(f"    Ref path:    {ref_path}")
        continue

    # Convert to grayscale and float32 (for calculations)
    sample = cv2.cvtColor(sample_color, cv2.COLOR_BGR2GRAY).astype(np.float32) # I
    ref    = cv2.cvtColor(ref_color, cv2.COLOR_BGR2GRAY).astype(np.float32)    # I0

    # Small value to avoid divide-by-zero or log(0)
    epsilon = 1e-6

    # === Calculate Transmittance (T = I / I0) ===
    transmittance = sample / (ref + epsilon)
    transmittance = np.clip(transmittance, epsilon, 1.0 - epsilon) # Clip for valid log range

    # === Compute Absorbance (A = -ln(T)) ===
    absorbance = -np.log(transmittance)

    # Normalize absorbance to 0-255 uint8 for consistent display with matplotlib
    abs_display = cv2.normalize(absorbance, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    # === Create and save the Matplotlib plot (without colorbar) ===
    plt.figure(figsize=(7, 6)) # Adjusted figsize slightly as colorbar is removed

    plt.imshow(abs_display, cmap='inferno') # Use 'inferno' colormap
    #plt.title(f'Absorbance Map - {wavelength}') # Dynamic title with wavelength
    # plt.colorbar(label='Normalized Absorbance Value') # Colorbar is commented out
    plt.axis('off') # Turn off axes for a clean appearance

    plt.tight_layout() # Adjust layout

    # === Save the Matplotlib plot ===
    # This line uses the 'wavelength' variable directly in the filename
    plot_filename = f"absorbance_map_plot_{wavelength}.png"
    plot_save_path = os.path.join(output_plot_folder, plot_filename)
    plt.savefig(plot_save_path, bbox_inches='tight', dpi=300) # Save the plot with high quality
    print(f"✅ Saved Absorbance Map plot to: {plot_save_path}")

    plt.show() # Display the plot window (optional)
    plt.close() # Close the plot window to free up memory